# Récupération des matchs manquants

Suite au merge des datasets nous avons remarqué qu'il manquait un grand nombre de d'observations. 

Certain matchs n'étaient pas dans les datasets play by play. Il est possible de les récupérer via l'API nba_api afin de les réinjecter dans notre pre processing

In [ ]:
# Matchs qui n'apparaissent que dans Shot locations
game_ids = df_merged[df_merged['GAME_ID'].isna()]['Game ID'].unique()
len(game_ids)

1714

In [ ]:
from nba_api.stats.endpoints import playbyplayv2
from tqdm import tqdm

# pour chaque game id de Shot locations non présent dans les play by play
# on récupère les play by play sur le site de la NBA
liste_df = []
for id in tqdm(game_ids):
    string_id = "00" + str(int(id))
    liste_df.append(playbyplayv2.PlayByPlayV2(string_id).get_data_frames()[0])


100%|██████████| 1714/1714 [31:03<00:00,  1.09s/it]


In [ ]:
# concaténation
missing_play_by_play = pd.concat(liste_df)

In [ ]:
# enregistrement du dataset
missing_play_by_play.to_csv("../data/raw/missing_pbp.csv")
